## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 12s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [10]:

classifier=Sequential()

input_shape = (32,32,3)

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32, kernel_size = (3,3), input_shape = input_shape, activation = 'relu'))
classifier.add(BatchNormalization(input_shape = input_shape))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size = (3,3), activation = 'relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units = 100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units = 10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 15s 30ms/step - loss: 1.3497 - accuracy: 0.5459
Epoch 2/100
500/500 [==============================] - 16s 32ms/step - loss: 0.8315 - accuracy: 0.7107
Epoch 3/100
500/500 [==============================] - 15s 30ms/step - loss: 0.6052 - accuracy: 0.7891
Epoch 4/100
500/500 [==============================] - 15s 30ms/step - loss: 0.4285 - accuracy: 0.8515
Epoch 5/100
500/500 [==============================] - 15s 30ms/step - loss: 0.2913 - accuracy: 0.8989
Epoch 6/100
500/500 [==============================] - 15s 31ms/step - loss: 0.2006 - accuracy: 0.9318
Epoch 7/100
500/500 [==============================] - 15s 30ms/step - loss: 0.1480 - accuracy: 0.9496
Epoch 8/100
500/500 [==============================] - 15s 30ms/step - loss: 0.1320 - accuracy: 0.9545
Epoch 9/100
500/500 [==============================] - 15s 30ms/step - loss: 0.1120 - accuracy: 0.9620
Epoch 10/100
500/500 [==============================] - 15s 30ms/step - l

500/500 [==============================] - 13s 26ms/step - loss: 0.0202 - accuracy: 0.9937
Epoch 80/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0085 - accuracy: 0.9972
Epoch 81/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0121 - accuracy: 0.9965
Epoch 82/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0156 - accuracy: 0.9952
Epoch 83/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0217 - accuracy: 0.9929
Epoch 84/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0163 - accuracy: 0.9952
Epoch 85/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0131 - accuracy: 0.9961
Epoch 86/100
500/500 [==============================] - 13s 27ms/step - loss: 0.0097 - accuracy: 0.9970
Epoch 87/100
500/500 [==============================] - 14s 27ms/step - loss: 0.0107 - accuracy: 0.9968
Epoch 88/100
500/500 [==============================] - 13s 27ms/step - loss:

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [11]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[5.5011591e-09, 3.5596626e-20, 4.0669157e-08, 9.9999976e-01,
        5.2589810e-10, 7.9423302e-18, 5.1605355e-08, 3.3819956e-18,
        1.4783539e-07, 2.7547635e-31]], dtype=float32)